In [1]:
!pip install -q git+https://github.com/google-research/vision_transformer

Reshimming asdf python...


In [33]:
import jax
import jax.numpy as jnp
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm

from vit_jax import models


model_name = 'LiT-B16B'

lit_model = models.get_model(model_name)
lit_variables = lit_model.load_variables()
tokenizer = lit_model.get_tokenizer()
image_preprocessing = lit_model.get_image_preprocessing()
pp = lit_model.get_pp()

import requests
from PIL import Image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = image.convert('RGB')
image_array = np.array(image)

image_array = np.expand_dims(image_array, axis=0) # add batch dimension
image = image_preprocessing(image_array)

texts = [
    'an image of cats on a couch',
    'a diagram of scrambled eggs',
    'many remote control devices spread out on a table',
    'a bad illustration of colorful houses',
]
tokens = tokenizer(texts)


"""
i have no idea if this makes sense, i just tried to figure out what the demo was doing
"""

_, ztxt, _ = lit_model.apply(lit_variables, tokens=tokens)
zimg, _, _ = lit_model.apply(lit_variables, images=image)

# normalize the embeddings
zimg_norm = zimg / jnp.linalg.norm(zimg, axis=-1, keepdims=True)
ztxt_norm = ztxt / jnp.linalg.norm(ztxt, axis=-1, keepdims=True)

# compute similarity scores between the image and texts
similarity_scores = jnp.dot(zimg_norm, ztxt_norm.T).squeeze()

print("similarity scores:")
for text, score in zip(texts, similarity_scores):
    print(f"\t{text}: {score * 100:.2f}%")

most_similar_index = jnp.argmax(similarity_scores)
most_similar_text = texts[most_similar_index]
highest_score = similarity_scores[most_similar_index]
print(f"\nmax similarity: '{most_similar_text}'")



⚠️ Reusing local copy: LiT-B16B.npz
similarity scores:
	an image of cats on a couch: 22.68%
	a diagram of scrambled eggs: 7.44%
	many remote control devices spread out on a table: 12.63%
	a bad illustration of colorful houses: 7.28%

max similarity: 'an image of cats on a couch'
